In [8]:
import evolvepy as ep
import numpy as np
import matplotlib.pyplot as plt

## Definindo a função a ser avaliada

In [9]:
# digite a sua expressão lógica usando + como AND, * como OR e ! como NOT
expression = "("
# digite o número de variáveis (não repetidas) que você usou na expressão
n_variables = 3

In [10]:
def fitness_function(individuals):
	individual = individuals[0]["chr0"] 
	score = 0
	individual += 1
	if individual[0] or individual[1] or individual[2] == True:
		score+=1
	if not individual[0] or individual[1] or individual[2] == True:
		score+=1

	return score

In [11]:
from evolvepy.evaluator import FunctionEvaluator

evaluator = FunctionEvaluator(fitness_function)

# Generator

Here we will define the generator for the problem

first, we need to define a descriptor with a boolean (true/false) chromossome

In [12]:
# here we have individual with 1 chromossome with type boolean. There is no need t define the chromossome varition range
descriptor = ep.generator.Descriptor(n_variables, types = bool)

In [13]:
print(descriptor.dtype)

[('chr0', '?', (3,))]


In [14]:
from evolvepy.generator import CombineLayer, Generator, FirstGenLayer
from evolvepy.generator.mutation import BinaryMutationLayer, bit_mutation
from evolvepy.generator.crossover import one_point
from evolvepy.generator.selection import tournament

combine = CombineLayer(tournament, one_point)
mutation = BinaryMutationLayer(bit_mutation, 0.8, 0.4, (0.0, 1.0))
first_layer = first_gen0 = FirstGenLayer(descriptor)
gen = Generator(descriptor=descriptor, first_layer=first_layer, last_layer=mutation)

AttributeError: 'BinaryMutationLayer' object has no attribute '_descriptor'

# Run and Results

Let's evolve our population and see the results

In [ ]:
n_generations = 50
evolver = ep.Evolver(gen, evaluator, n_individual)
hist, last_population = evolver.evolve(n_generations)

NameError: name 'cache' is not defined